In [90]:
# import code block
from selenium import webdriver
from bs4 import BeautifulSoup
import zipfile
import pandas as pd
import csv
import time
import random
import matplotlib.pyplot as plt
from urllib.request import urlopen
import re
import string
import numpy as np
import sys
from datetime import date
import datetime
from statsmodels.tsa.arima.model import ARIMA
from random import random

In [57]:
evictionLab = pd.read_csv('tracts.csv')
evictionLab.rename(columns={'name':'CETRACT'}, inplace=True)
evictionLab.head(25)

,GEOID,year,CETRACT,parent-location,population,poverty-rate,renter-occupied-households,pct-renter-occupied,median-gross-rent,median-household-income,...,pct-nh-pi,pct-multiple,pct-other,eviction-filings,evictions,eviction-rate,eviction-filing-rate,low-flag,imputed,subbed
0,25001010100,2000,101.00,"Barnstable County, Massachusetts","3,433.09",16.28,849,46.20,644.01,"32,722.44",...,0.00,2.88,0.82,nan,nan,nan,nan,0,0,0
1,25001010100,2001,101.00,"Barnstable County, Massachusetts","3,433.09",16.28,842,46.20,644.01,"32,722.44",...,0.00,2.88,0.82,nan,nan,nan,nan,0,0,0
2,25001010100,2002,101.00,"Barnstable County, Massachusetts","3,433.09",16.28,836,46.20,644.01,"32,722.44",...,0.00,2.88,0.82,nan,nan,nan,nan,0,0,0
3,25001010100,2003,101.00,"Barnstable County, Massachusetts","3,433.09",16.28,829,46.20,644.01,"32,722.44",...,0.00,2.88,0.82,31.00,30.00,3.62,3.74,0,0,0
4,25001010100,2004,101.00,"Barnstable County, Massachusetts","3,433.09",16.28,822,46.20,644.01,"32,722.44",...,0.00,2.88,0.82,nan,nan,nan,nan,0,0,0
5,25001010100,2005,101.00,"Barnstable County, Massachusetts","3,409.16",0.00,816,31.68,878.20,"43,969.25",...,0.00,2.99,0.00,nan,nan,nan,nan,0,0,0
6,25001010100,2006,101.00,"Barnstable County, Massachusetts","3,409.16",0.00,809,31.68,878.20,"43,969.25",...,0.00,2.99,0.00,nan,nan,nan,nan,0,0,0
7,25001010100,2007,101.00,"Barnstable County, Massachusetts","3,409.16",0.00,802,31.68,878.20,"43,969.25",...,0.00,2.99,0.00,nan,nan,nan,nan,0,0,0
8,25001010100,2008,101.00,"Barnstable County, Massachusetts","3,409.16",0.00,795,31.68,878.20,"43,969.25",...,0.00,2.99,0.00,nan,nan,nan,nan,0,0,0
9,25001010100,2009,101.00,"Barnstable County, Massachusetts","3,409.16",0.00,789,31.68,878.20,"43,969.25",...,0.00,2.99,0.00,nan,nan,nan,nan,0,0,0


In [3]:
forecasting_df = evictionLab[['CETRACT', 'year','evictions', 'eviction-rate']].copy()
list_of_cetracts = forecasting_df['CETRACT'].unique()

forecasting_df = forecasting_df.fillna(0)
forecasting_df = forecasting_df.reset_index()
forecasting_df = forecasting_df.drop(['index'], axis=1)

forecasting_df.head(20)

,CETRACT,year,evictions,eviction-rate
0,101.00,2000,0.0,0.00
1,101.00,2001,0.0,0.00
2,101.00,2002,0.0,0.00
3,101.00,2003,30.0,3.62
4,101.00,2004,0.0,0.00
5,101.00,2005,0.0,0.00
6,101.00,2006,0.0,0.00
7,101.00,2007,0.0,0.00
8,101.00,2008,0.0,0.00
9,101.00,2009,0.0,0.00


In [25]:
# build dictionary
eviction_dict = {}

for index, row in forecasting_df.iterrows():
    if row['CETRACT'] not in eviction_dict:
        eviction_dict[row['CETRACT']] = [[row['year'], row['evictions'], row['eviction-rate']]]
    else:
        eviction_dict[row['CETRACT']].append([row['year'], row['evictions'], row['eviction-rate']])


In [56]:
# dictionary format is: Key == CETRACT, and each key maps to a 2D array where each list item is in the form of 
# [year, # of evictions, and eviction rate]
# large dictionary, so commenting out print statement below, if you want to see contents just uncomment and run code block
# print(eviction_dict)

In [52]:
# x is the list of data (i.e. evictions) and w is the window we want to use to predict the next value
# returns the average of the last w items, which will be used as the predicted value
def moving_average(x, w):
    sum = 0
    for x in x[-w:]:
        sum += x
    return sum//w 

def predict_from_moving_average(x):
    # contrived dataset
    data = x
    # fit model
    model = ARIMA(data, order=(0, 0, 1))
    model_fit = model.fit()
    # make prediction
    yhat = model_fit.predict(len(data), len(data))
    return yhat


In [26]:
# generalize for whole dictionary
for key in eviction_dict:
    # save the dictionary that this key points to
    current_ctract_dict = eviction_dict[key]
    # lists to keep track of year, evictions, and eviction rate
    years = []
    evictions = []
    eviction_rate = []
    #build our lists
    for i in range(len(current_ctract_dict)):
        years.append(int(current_ctract_dict[i][0]))
        evictions.append(int(current_ctract_dict[i][1]))
        eviction_rate.append(current_ctract_dict[i][2])
    # add predictions for the next 4 years that are missing (2017 - 2020)
    # and append to the lists we made above
    for i in range(4):
        years.append(years[-1] + 1)
        evictions.append(int(predict_from_moving_average(evictions)))
        eviction_rate.append(float(predict_from_moving_average(eviction_rate)))
    # update dictionary values
    for i in range(17, 21):
        eviction_dict[key].append([years[i], evictions[i], eviction_rate[i]])
    

/Users/gonzo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
/Users/gonzo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
/Users/gonzo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
/Users/gonzo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
/Users/gonzo/opt/anaconda3/lib/python3.8/site-pa

/Users/gonzo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
/Users/gonzo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
/Users/gonzo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
/Users/gonzo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
/Users/gonzo/opt/anaconda3/lib/python3.8/site-pa

/Users/gonzo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
/Users/gonzo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
/Users/gonzo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
/Users/gonzo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
/Users/gonzo/opt/anaconda3/lib/python3.8/site-pa

/Users/gonzo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
/Users/gonzo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
/Users/gonzo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
/Users/gonzo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
/Users/gonzo/opt/anaconda3/lib/python3.8/site-pa

In [89]:
# print(eviction_dict)

In [96]:
# build back dataframe
data = []
# print(eviction_dict[101.0])
# print(len(eviction_dict[101.0]))
# print(eviction_dict[101.0])
for ctract in eviction_dict:
    for list in eviction_dict[ctract]:
        data.append([ctract, list[0], list[1], list[2]])


In [97]:
evictions_with_forecast = pd.DataFrame(data, columns = ['CETRACT', 'year','number_of_evictions','eviction_rate'])

# reformat data
pd.options.display.float_format = "{:,.2f}".format
data_types_dict = {'CETRACT': float,'year':int,'number_of_evictions':int,'eviction_rate':float}
evictions_with_forecast = evictions_with_forecast.astype(data_types_dict)


In [98]:
GEOID_df = evictionLab[['GEOID','CETRACT']].copy()


In [99]:
# merge to get GEOID values
evictions_with_forecast = pd.merge(evictions_with_forecast, GEOID_df, on='CETRACT', how='right')
evictions_with_forecast = evictions_with_forecast.drop_duplicates(subset=None, keep='first', inplace=False)
evictions_with_forecast = evictions_with_forecast.reset_index()
evictions_with_forecast = evictions_with_forecast.drop(['index'], axis=1)
evictions_with_forecast.head(25)

,CETRACT,year,number_of_evictions,eviction_rate,GEOID
0,101.00,2000,0,0.00,25001010100
1,101.00,2001,0,0.00,25001010100
2,101.00,2002,0,0.00,25001010100
3,101.00,2003,30,3.62,25001010100
4,101.00,2004,0,0.00,25001010100
5,101.00,2005,0,0.00,25001010100
6,101.00,2006,0,0.00,25001010100
7,101.00,2007,0,0.00,25001010100
8,101.00,2008,0,0.00,25001010100
9,101.00,2009,0,0.00,25001010100


In [88]:
# export as CSV
evictions_with_forecast.to_csv('eviction_data_with_forecast_vals.csv')
